In [1]:
import tensorflow
from tensorflow.keras.models import Sequential,load_model
import pickle
import numpy as np
import datetime
import pandas as pd

In [3]:
# load the trained model
model = load_model('model.h5')

# load the pickle files for inference later
with open('scaler.pkl','rb') as f:
    scaler = pickle.load(f)
with open('label_encoder_gender.pkl','rb') as f:
    label_encoder_gender = pickle.load(f)
with open('one_hot_encoder_geo.pkl','rb') as f:
    label_encoder_geo = pickle.load(f)

In [4]:

input_data = {
'CreditScore':600,
'Geography': 'France',
'Gender': 'Male',
'Age': 40,
'Tenure': 3,
'Balance': 60000,
'NumOfProducts': 2,
'HasCrCard': 1,
'IsActiveMember': 1,
'EstimatedSalary': 50000
}
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [5]:
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])

In [6]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [7]:
# use the loaded pickle files to preprocess the input data
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))

d:\Generative AI\Projects\AnnClassify\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [8]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [9]:
# concat the encoded geography columns with the original dataframe
input_df = pd.concat([input_df.drop(columns=['Geography']), geo_encoded_df], axis=1)

In [10]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [11]:
# scale the input data
input_data_scaled = scaler.transform(input_df)

In [12]:
#  predict using the trained model
prediction = model.predict(input_data_scaled)
prediction_class = (prediction > 0.5).astype("int32")
prediction_probability = prediction[0][0]
print(f"Churn Probability: {prediction_probability:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
Churn Probability: 0.2425


In [16]:
prediction[0][0]
prediction_class

array([[0]], dtype=int32)

In [17]:
if prediction_probability > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is unlikely to churn.")

The customer is unlikely to churn.
